<h1>Installs</h1>
- This file prepares the data such that we can upload it to opensearch (So that we can have vecotrs and metadata all in one DB). We use the same e5 model as in strategy_1 but chunk the data with the SentenceTransformersTokenTextSplitter method. 

In [1]:
! pip install pinecone-client
! pip install faunadb
! pip install ndg-httpsclient
! pip install pyopenssl
! pip install pyasn1
! pip install -U sentence-transformers
! pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because norma


<h1>Extract Data from .csv</h1>
*   AB = Column that corresponds to the text of the abstract


In [2]:
import csv
from itertools import islice
import uuid

# Load data, store abstract text
batch_input = []
with open('../data/INLPT_class/articles.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        batch_input.append([row["AB"],row["PMID"],row["TI"],row["PB"],row["FAU"],row["FED"],row["DP"],row["OTO"], row["OT"],row["OWN"], row["LR"], row["JT"], row["MH"],row["ISBN"]])

In [40]:
print(len(batch_input))

57560


In [ ]:

<h1>Embedd Data from .csv</h1>
1. DIvide each abstract into chunks such that each chunks text will be tokenized to max 512 Tokens


In [3]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
import re
import tqdm

#embed data
# model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
# tokenizer = AutoTokenizer.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')
# max_token_size = 350
# print(model)
# print(tokenizer)

model_name = 'intfloat/e5-base-v2'
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_token_size = 512
print(model)
print(tokenizer)


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)
BertTokenizerFast(name_or_path='intfloat/e5-base-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),


In [4]:
splitter = SentenceTransformersTokenTextSplitter(
    model_name=model_name,  
    chunk_overlap=20,  
)
print(splitter.maximum_tokens_per_chunk)

512


In [5]:
chunked_data = []

#Chunk abstract text, prepare for emebdding
for abstract_text in batch_input:
    chunked_data.append(splitter.split_text(text=abstract_text[0]))

2024-01-12 01:09:28.944477: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 01:09:30.184028: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 01:09:30.184064: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 01:09:30.184094: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 01:09:30.722738: I tensorflow/core/platform/cpu_feature_g

In [33]:
print(len(chunked_data))

57560


In [6]:
chunks = []

#flatten data and append metadata to each chunk -> list of [chunk, PMID, TI, PB, FAU, FED,DP,OTO,ISBN]
for id, abstract in enumerate(chunked_data):
    for chunk in abstract:
        chunks.append([chunk, batch_input[id][1],batch_input[id][2],batch_input[id][3],batch_input[id][4],batch_input[id][5],batch_input[id][6],batch_input[id][7],batch_input[id][8],batch_input[id][9],batch_input[id][10],batch_input[id][11],batch_input[id][12]])

In [44]:
print(len(chunks))

63398


In [7]:
from tqdm import tqdm

In [ ]:
#add embedding vector of chunk to the list
for chunk in tqdm(chunks):
    chunk.insert(0, model.encode(chunk[0]).tolist())

 80%|███████▉  | 50546/63398 [09:46<02:30, 85.37it/s]

In [ ]:
import pickle

#save List in File. This file will be later accesssed and the data will be uploaded to open search

path = "./data/data.txt"
with open(path, 'wb') as file:
    pickle.dump(chunks, file)